# Image Classification menggunakan CNN

#### Kaggle dataset

In [ ]:
!pip install kaggle

In [ ]:
!kaggle datasets download -d rendyys/dataset-penyakit-kulit

print('Data source import complete.')

In [ ]:
!unzip penyakit-kulit.zip

In [ ]:
folder_path = "penyakit-kulit"
file_list = os.listdir(folder_path)
print("Files in the folder:", file_list)

#### Drive dataset

File ada pada [link ini](https://drive.google.com/drive/folders/129u1wP2X6AVIrScrxAyvxZzDh-Y1uNsf?usp=drive_link)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import PIL
import pathlib
from google.colab import files
from PIL import Image
import imghdr

from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,Dropout, Flatten,Activation, BatchNormalization,MaxPooling2D
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.metrics import Precision, Accuracy, Recall

In [ ]:
folder_path = "/content/drive/MyDrive/penyakit-kulit/"
file_list = os.listdir(folder_path)
print("Files in the folder:", file_list)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator (rescale=1/255)
test_datagen = ImageDataGenerator (rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/penyakit-kulit/train',
    classes = ['acne','eksim','herpes','panu','rosacea'],
    target_size=(200, 200),
    batch_size=16)

validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/penyakit-kulit/valid',
    classes = ['acne','eksim','herpes','panu','rosacea'],
    target_size=(200, 200),
    batch_size=16,
    shuffle=False)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/penyakit-kulit/test',
    classes = ['acne','eksim','herpes','panu','rosacea'],
    target_size=(200, 200),
    batch_size=16,
    shuffle=False)


In [ ]:
model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(200, 200, 3)))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(5, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics= [Recall(), Precision(), "accuracy"]
)

In [ ]:
model_callback = EarlyStopping(
    monitor = 'loss',
    verbose = 1,
    mode = 'min',
    patience= 3
)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=75,
    steps_per_epoch=100,
    callbacks = [model_callback],
    verbose=1
)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/penyakit-kulit/test',
    classes = ['acne','eksim','herpes','panu','rosacea'],
    target_size=(200, 200),
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
model.evaluate(test_generator)

In [ ]:
plt.figure(figsize=(8, 3))
plt.plot(history.epoch, history.history['accuracy'])
plt.plot(history.epoch, history.history['val_accuracy'])
plt.legend(['train accu', 'val accu'])
plt.title('Accuracy Diagram')
plt.xlabel('Epoch(s)')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
plt.figure(figsize=(8, 3))
plt.plot(history.epoch, history.history['loss'])
plt.plot(history.epoch, history.history['val_loss'])
plt.legend(['train loss', 'val loss'])
plt.title('Loss Diagram')
plt.xlabel('Epoch(s)')
plt.ylabel('Loss')
plt.show()

In [ ]:
predictions = model.predict(test_generator, steps=len(test_generator), verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
cm = confusion_matrix(true_classes, predicted_classes)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=test_generator.class_indices.keys())
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

report = classification_report(true_classes, predicted_classes, target_names=test_generator.class_indices.keys())
print(report)

In [ ]:
width = 200
height = 200
classes = ['acne','eksim','herpes','panu','rosacea']
fileImage = Image.open("/content/drive/MyDrive/penyakit-kulit/valid/acne/ac86_JPG.rf.e5a4d64f2c42bd1e7cb5663bc782e1fd.jpg").convert("RGB").resize([width, height], Image.LANCZOS)
image = np.array(fileImage)
myimage = image.reshape(1, width, height, 3)
plt.figure(figsize=(4, 2))
plt.imshow(image)

my_predicted_image = model.predict(myimage)
y_class = np.argmax(my_predicted_image)
print("Class:", y_class, "Name:", classes[y_class])

In [ ]:
fileImage = Image.open("/content/drive/MyDrive/penyakit-kulit/valid/rosacea/ro22_jpg.rf.88f2d4f4e04a3f3583df0672b87fef42.jpg").convert("RGB").resize([width, height], Image.LANCZOS)
image = np.array(fileImage)
myimage = image.reshape(1, width, height, 3)
plt.figure(figsize=(4, 2))
plt.imshow(image)

my_predicted_image = model.predict(myimage)
y_class = np.argmax(my_predicted_image)
print("Class:", y_class, "Name:", classes[y_class])

In [ ]:
fileImage = Image.open("/content/drive/MyDrive/penyakit-kulit/valid/panu/pa26_jpg.rf.fd2fe80d03604f654b01d179789a937d.jpg").convert("RGB").resize([width, height], Image.LANCZOS)
image = np.array(fileImage)
myimage = image.reshape(1, width, height, 3)
plt.figure(figsize=(4, 2))
plt.imshow(image)

my_predicted_image = model.predict(myimage)
y_class = np.argmax(my_predicted_image)
print("Class:", y_class, "Name:", classes[y_class])

In [ ]:
fileImage = Image.open("/content/drive/MyDrive/penyakit-kulit/valid/herpes/he49_jpg.rf.084fba655e2b89c68b5e7d6aaa312974.jpg").convert("RGB").resize([width, height], Image.LANCZOS)
image = np.array(fileImage)
myimage = image.reshape(1, width, height, 3)
plt.figure(figsize=(4, 2))
plt.imshow(image)

my_predicted_image = model.predict(myimage)
y_class = np.argmax(my_predicted_image)
print("Class:", y_class, "Name:", classes[y_class])

Lakukan predict ke gambar baru

In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():
    path = '/content/' + fn
    img = image.load_img(path, target_size=(200, 200))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    classes = model.predict(img_array / 255.0)
    predicted_class_index = np.argmax(classes[0])
    class_names = ["acne", "eksim", "herpes", "panu", "rosacea"]
    predicted_class_name = class_names[predicted_class_index]
    print(f"Predicted class: {predicted_class_name}")

    plt.figure(figsize=(4, 2))
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    print(f"Class index: {predicted_class_index}, Class Name: {predicted_class_name}")

In [ ]:
#Save Model
model.save('model_dt_kulit.h5')